In [ ]:
from io import BytesIO
import onnx
import torch
from ultralytics import YOLO
from gdrive.MyDrive.Quantization.TensorRT.models.common import PostDetect, optim
from gdrive.MyDrive.Quantization.TensorRT.models import EngineBuilder
import onnxsim

#### pt to onnx

In [ ]:
pt_model = 'yolov8m.pt'

PostDetect.conf_thres = 0.25
PostDetect.iou_thres = 0.65
PostDetect.topk = 100

b = 1
YOLOv8 = YOLO(pt_model)
model = YOLOv8.model.fuse().eval()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
for m in model.modules():
    optim(m)
    m.to(device)
model.to(device)
fake_input = torch.randn((1,3,640,640)).to(device)
for _ in range(2):
    model(fake_input)
save_path = pt_model.replace('.pt', '.onnx')
with BytesIO() as f:
    torch.onnx.export(
        model,
        fake_input,
        f,
        opset_version=11,
        input_names=['images'],
        output_names=['num_dets', 'bboxes', 'scores', 'labels'])
    f.seek(0)
    onnx_model = onnx.load(f)
onnx.checker.check_model(onnx_model)
shapes = [b, 1, b, 100, 4, b, 100, b, 100]
for i in onnx_model.graph.output:
    for j in i.type.tensor_type.shape.dim:
        j.dim_param = str(shapes.pop(0))
try:
    onnx_model, check = onnxsim.simplify(onnx_model)
    assert check, 'assert check failed'
except Exception as e:
    print(f'Simplifier failure: {e}')
onnx.save(onnx_model, save_path)
print(f'ONNX export success, saved as {save_path}')

#### Onnx to tensorrt float16

In [ ]:
onnx_model = 'gdrive/MyDrive/yolov8m.onnx'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

builder = EngineBuilder(onnx_model, device)
builder.seg = True
builder.weight = 'gdrive/MyDrive/FP16.engine'
builder.build(precision='int',
              input_shape=[1, 3, 640, 640],
              iou_thres=0.65,
              conf_thres=0.25,
              topk=100)

In [ ]:
import cv2
import numpy as np
import random
from gdrive.MyDrive.Quantization.TensorRT.models.utils import blob, det_postprocess, letterbox
from gdrive.MyDrive.Quantization.TensorRT.models.cudart_api import TRTEngine

engine_path = 'FP16.engine'
Engine = TRTEngine(engine_path)
H, W = Engine.inp_info[0].shape[-2:]

cap = cv2.VideoCapture('video.mp4')

if (cap.isOpened()== False):
    print("Error opening video stream or file")

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        draw = frame.copy()
        bgr, ratio, dwdh = letterbox(frame, (W, H))
        rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
        tensor = blob(rgb, return_seg=False)
        dwdh = np.array(dwdh * 2, dtype=np.float32)
        tensor = np.ascontiguousarray(tensor)
        # inference
        data = Engine(tensor)
        bboxes, scores, labels = det_postprocess(data)
        bboxes -= dwdh
        bboxes /= ratio

        for (bbox, score, label) in zip(bboxes, scores, labels):
            bbox = bbox.round().astype(np.int32).tolist()
            cls_id = int(label)
            cls = 'face'
            color = (255, 0, 0)
            cv2.rectangle(draw, bbox[:2], bbox[2:], color, 2)
            cv2.putText(draw, f'{cls}:{score:.3f}', (bbox[0], bbox[1] - 2), cv2.FONT_HERSHEY_SIMPLEX, 0.75, [225, 255, 255], thickness=2)

        cv2.imshow('video', draw)

    # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Onnx to tensorrt float32

In [ ]:
onnx_model = 'yolov8m.onnx'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

builder = EngineBuilder(onnx_model, device)
builder.seg = True
builder.weight = 'FP32.engine'
builder.build(precision='fp32',
              input_shape=[1, 3, 640, 640],
              iou_thres=0.65,
              conf_thres=0.25,
              topk=100)

Onnx to tensorrt int8

In [ ]:
onnx_model = 'yolov8m.onnx'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

builder = EngineBuilder(onnx_model, device)
builder.seg = True
builder.weight = 'INT8.engine'
builder.build(precision='int8',
              input_shape=[1, 3, 640, 640],
              iou_thres=0.65,
              conf_thres=0.25,
              topk=100)